# Install

In [3]:
%load_ext autoreload
%autoreload 2
    
from circuits_benchmark.utils.project_paths import detect_project_root
print(detect_project_root())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/workspace/circuits-benchmark


In [19]:
# !rm /workspace/pyproject.toml
! ls /workspace

Dockerfile  circuits-benchmark	    project_template
README.md   cluster_setup.egg-info  quick_experiments


In [1]:
import sys
import importlib.util
import importlib.machinery

# Specify the directory with your local package
local_directory = "/workspace/circuits-benchmark"

# The package name
package_name = "tracr"

# Construct the full path to the package's __init__.py file
package_init_file = f"{local_directory}/{package_name}/__init__.py"

# Load the package from the specified file
spec = importlib.util.spec_from_file_location(package_name, package_init_file)
local_package = importlib.util.module_from_spec(spec)
sys.modules[package_name] = local_package
spec.loader.exec_module(local_package)

# Now you can use the tracr package as usual
import tracr

# Example usage of tracr
print(tracr.__file__)
from tracr.compiler.compiling import TracrOutput

/workspace/circuits-benchmark/tracr/__init__.py


In [2]:
import circuits_benchmark.benchmark.cases.case_3 as case_3

In [17]:
from circuits_benchmark.utils.project_paths import detect_project_root
from circuits_benchmark.utils.project_paths import PROJECT_ROOT
def update_proj_root():
    global PROJECT_ROOT
    print(PROJECT_ROOT)
    PROJECT_ROOT = "/workspace/circuits-benchmark"
    print(PROJECT_ROOT)
update_proj_root()
print(detect_project_root())

# import os


# PROJECT_ROOT: str | None = None


# def detect_project_root() -> str:
#   """
#   Detects the root of the project by looking for a known file in the project.
#   :return: the path to the root of the project.
#   """
#   global PROJECT_ROOT
#   if PROJECT_ROOT is not None:
#     # If the project root has already been detected, return it.
#     return PROJECT_ROOT

#   current_path = os.path.abspath(os.path.curdir)
#   while not os.path.exists(os.path.join(current_path, "pyproject.toml")):
#     print(current_path)
#     current_path = os.path.abspath(os.path.join(current_path, os.pardir))

#   PROJECT_ROOT = current_path

#   return current_path

# print(detect_project_root())

None
/workspace/circuits-benchmark


KeyboardInterrupt: 

In [8]:
!git lfs install
!git clone https://huggingface.co/cybershiptrooper/InterpBench

Updated git hooks.
Git LFS initialized.
Cloning into 'InterpBench'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 225 (delta 71), reused 0 (delta 0), pack-reused 4 (from 1)
Receiving objects: 100% (225/225), 385.59 KiB | 3.44 MiB/s, done.
Resolving deltas: 100% (71/71), done.
Filtering content: 100% (55/55), 82.13 MiB | 34.38 MiB/s, done.


# Imports and setup

In [4]:
import pickle
import torch
import os
import shutil
from transformer_lens import HookedTransformerConfig, HookedTransformer
from transformer_lens import HookedTransformer
from circuits_benchmark.transformers.hooked_tracr_transformer import HookedTracrTransformer

from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, decoders, trainers
from transformers import PreTrainedTokenizerFast


from sae_lens import LanguageModelSAERunnerConfig, SAETrainingRunner
from sae_lens import SAEConfig, SAE, TrainingSAEConfig, TrainingSAE, ActivationsStore, CacheActivationsRunnerConfig, LanguageModelSAERunnerConfig
from sae_lens.training.sae_trainer import SAETrainer

import sae_lens
print(dir(sae_lens.training.sae_trainer))

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

['ActivationsStore', 'Adam', 'Any', 'FINETUNING_PARAMETERS', 'HookedRootModule', 'L1Scheduler', 'LanguageModelSAERunnerConfig', 'SAETrainer', 'TrainSAEOutput', 'TrainStepOutput', 'TrainingSAE', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', '_log_feature_sparsity', '_update_sae_lens_training_version', 'cast', 'contextlib', 'dataclass', 'get_lr_scheduler', 'run_evals', 'torch', 'tqdm', 'wandb']
Using device: cuda


In [5]:
import json
from safetensors.torch import save_file
import wandb
from sae_lens.sae import SAE_CFG_PATH, SAE_WEIGHTS_PATH, SPARSITY_PATH

def save_checkpoint(
        trainer: SAETrainer,
        checkpoint_name: int | str,
        wandb_aliases: list[str] | None = None,
    ) -> str:
        """ Lightly modified from https://github.com/jbloomAus/SAELens/blob/v3.5.0/sae_lens/sae_training_runner.py#L161C5-L210C31 """

        sae = trainer.sae
        os.makedirs(trainer.cfg.checkpoint_path, exist_ok=True)
        checkpoint_path = f"{trainer.cfg.checkpoint_path}/{checkpoint_name}"

        os.makedirs(checkpoint_path, exist_ok=True)

        path = f"{checkpoint_path}"
        os.makedirs(path, exist_ok=True)

        if sae.cfg.normalize_sae_decoder:
            sae.set_decoder_norm_to_unit_norm()
        sae.save_model(path)

        # let's over write the cfg file with the trainer cfg, which is a super set of the original cfg.
        # and should not cause issues but give us more info about SAEs we trained in SAE Lens.
        config = trainer.cfg.to_dict()
        with open(f"{path}/cfg.json", "w") as f:
            json.dump(config, f)
        if trainer.cfg.log_to_wandb:
            print(f'saving {path}')
            wandb.save(path)


        log_feature_sparsities = {"sparsity": trainer.log_feature_sparsity}

        log_feature_sparsity_path = f"{path}/{SPARSITY_PATH}"
        save_file(log_feature_sparsities, log_feature_sparsity_path)

        if trainer.cfg.log_to_wandb and os.path.exists(log_feature_sparsity_path):
            model_artifact = wandb.Artifact(
                f"{sae.get_name()}",
                type="model",
                metadata=dict(trainer.cfg.__dict__),
            )

            model_artifact.add_file(f"{path}/{SAE_WEIGHTS_PATH}")
            model_artifact.add_file(f"{path}/{SAE_CFG_PATH}")

            wandb.log_artifact(model_artifact, aliases=wandb_aliases)

            sparsity_artifact = wandb.Artifact(
                f"{sae.get_name()}_log_feature_sparsity",
                type="log_feature_sparsity",
                metadata=dict(trainer.cfg.__dict__),
            )
            sparsity_artifact.add_file(log_feature_sparsity_path)
            wandb.log_artifact(sparsity_artifact)

        return checkpoint_path

In [6]:
import circuits_benchmark.benchmark.cases.case_3 as case_3

task = case_3.Case3()
task_idx = task.get_index()

In [9]:
dir_name = f"InterpBench/{task_idx}"
cfg_dict = pickle.load(open(f"{dir_name}/ll_model_cfg.pkl", "rb"))
cfg = HookedTransformerConfig.from_dict(cfg_dict)
model = HookedTransformer(cfg)
weights = torch.load(f"{dir_name}/ll_model.pth")
model.load_state_dict(weights)

<All keys matched successfully>

# Load and configure benchmark

In [10]:
%cd /workspace/circuits-benchmark
# load high level model
from circuits_benchmark.utils.iit import make_iit_hl_model
import circuits_benchmark.utils.iit.correspondence as correspondence
from circuits_benchmark.utils.iit.dataset import get_unique_data
import iit.model_pairs as mp
from datasets import Dataset

def make_model_pair(benchmark_case):
    hl_model = benchmark_case.build_transformer_lens_model()
    hl_model = make_iit_hl_model(hl_model, eval_mode=True)
    tracr_output = benchmark_case.get_tracr_output()
    hl_ll_corr = correspondence.TracrCorrespondence.from_output(
            case=benchmark_case, tracr_output=tracr_output
        )
    model_pair = mp.StrictIITModelPair(hl_model, model, hl_ll_corr)
    return model_pair, hl_model

model_pair, hl_model = make_model_pair(task)

/opt/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspace/circuits-benchmark
Moving model to device:  cuda
{'hook_embed': HookPoint(), 'hook_pos_embed': HookPoint(), 'blocks.0.attn.hook_k': HookPoint(), 'blocks.0.attn.hook_q': HookPoint(), 'blocks.0.attn.hook_v': HookPoint(), 'blocks.0.attn.hook_z': HookPoint(), 'blocks.0.attn.hook_attn_scores': HookPoint(), 'blocks.0.attn.hook_pattern': HookPoint(), 'blocks.0.attn.hook_result': HookPoint(), 'blocks.0.mlp.hook_pre': HookPoint(), 'blocks.0.mlp.hook_post': HookPoint(), 'blocks.0.hook_attn_in': HookPoint(), 'blocks.0.hook_q_input': HookPoint(), 'blocks.0.hook_k_input': HookPoint(), 'blocks.0.hook_v_input': HookPoint(), 'blocks.0.hook_mlp_in': HookPoint(), 'blocks.0.hook_attn_out': HookPoint(), 'blocks.0.hook_mlp_out': HookPoint(), 'blocks.0.hook_resid_pre': HookPoint(), 'blocks.0.hook_resid_mid': HookPoint(), 'blocks.0.hook_resid_post': HookPoint(), 'blocks.1.attn.hook_k': HookPoint(), 'blocks.1.attn.hook_q': HookPoint(), 'blocks.1.attn.hook_v': HookPoint(), 'blocks.1.attn.hook_z': Ho

In [11]:
# Create dataset of case inputs
dataset = get_unique_data(task, max_len=10_000)
tokenized_data = hl_model.map_tracr_input_to_tl_input(dataset.data)

# Convert PyTorch tensors to lists
string_tokens_list = dataset.data.tolist()
tokens_list = tokenized_data.tolist()
labels_list = [str(label) for label in dataset.labels]

# Create a dictionary from the lists
data_dict = {
    "string_tokens": string_tokens_list,
    "tokens": tokens_list,
    "labels": labels_list
}

# Create a Hugging Face dataset
hf_dataset = Dataset.from_dict(data_dict)

print(hf_dataset)
print(hf_dataset[3])

Dataset({
    features: ['string_tokens', 'tokens', 'labels'],
    num_rows: 1280
})
{'string_tokens': ['BOS', 'b', 'a', 'a', 'x'], 'tokens': [0, 3, 2, 2, 5], 'labels': "['BOS', 0.0, 0.0, 0.0, 0.25]"}


In [12]:
print(hf_dataset.shape)

(1280, 3)


In [13]:
# create tokenizer
# Define your simple vocabulary
vocab = {'BOS': 0, 'UNK': 1, 'a': 2, 'b': 3, 'c': 4, 'x': 5}
# comes from task.get_vocab() and hl_model.map_tracr_input_to_tl_input

# Create a Tokenizer with a WordLevel model
tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token="UNK"))

# Set the normalizer, pre-tokenizer, and decoder
tokenizer.normalizer = normalizers.Sequence([normalizers.Lowercase(), normalizers.StripAccents()])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Convert to Hugging Face tokenizer
hf_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

# Add the special tokens to the Hugging Face tokenizer
hf_tokenizer.add_special_tokens({
    'unk_token': 'UNK',
    'bos_token': 'BOS',
    'cls_token': '[CLS]',
    'sep_token': '[SEP]',
    'pad_token': '[PAD]',
    'mask_token': '[MASK]'
})

# Test the tokenizer
encoded = hf_tokenizer.encode("BOS a b c x")
decoded = hf_tokenizer.decode(encoded)
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")

Encoded: [0, 2, 3, 4, 5]
Decoded: BOS a b c x


In [14]:
model.tokenizer = hf_tokenizer #attach to model.

In [15]:
_, cache = model.run_with_cache(tokenized_data)
output = hl_model(tokenized_data) #TODO: why are these different calls?
print(output[:5], dataset.labels[:5])

tensor([[[0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.0000],
         [1.0000],
         [0.5000],
         [0.6667],
         [0.5000]],

        [[0.0000],
         [0.0000],
         [0.0000],
         [0.0000],
         [0.2500]],

        [[0.0000],
         [0.0000],
         [0.0000],
         [0.3333],
         [0.2500]]], device='cuda:0') [list(['BOS', 0.0, 0.0, 0.0, 0.0]) list(['BOS', 0.0, 0.0, 0.0, 0.0])
 list(['BOS', 1.0, 0.5, 0.6666666666666666, 0.5])
 list(['BOS', 0.0, 0.0, 0.0, 0.25])
 list(['BOS', 0.0, 0.0, 0.3333333333333333, 0.25])]


In [16]:
print(cache)

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.hook_q_input', 'blocks.0.hook_k_input', 'blocks.0.hook_v_input', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.hook_result', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.hook_mlp_in', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.hook_q_input', 'blocks.1.hook_k_input', 'blocks.1.hook_v_input', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.attn.hook_result', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.hook_mlp_in', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post

# SAE-lens

In [17]:
%cd /content
%pwd
%ls

[Errno 2] No such file or directory: '/content'
/workspace/circuits-benchmark
DEMO_InterpBench.ipynb  circuits_benchmark/  pytest.ini
Dockerfile              iit@                 submodules/
EXPERIMENTS.md          main.py*             subnetwork_probing@
LICENSE                 metadata/            tracr@
README.md               poetry.lock
acdc@                   pyproject.toml


/opt/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [18]:
class RepeatActivationsStore(ActivationsStore):
    def _get_next_dataset_tokens(self) -> torch.Tensor:
        device = self.device
        if not self.is_dataset_tokenized:
            try:
                s = next(self.iterable_dataset)[self.tokens_column]
            except StopIteration:
                #shuffle self.dataset and restart
                self.iterable_dataset = iter(self.dataset)
                s = next(self.iterable_dataset)[self.tokens_column]
            tokens = (
                self.model.to_tokens(
                    s,
                    truncate=False,
                    move_to_device=True,
                    prepend_bos=self.prepend_bos,
                )
                .squeeze(0)
                .to(device)
            )
            assert (
                len(tokens.shape) == 1
            ), f"tokens.shape should be 1D but was {tokens.shape}"
        else:
            try:
                s = next(self.iterable_dataset)[self.tokens_column]
            except StopIteration:
                #shuffle self.dataset and restart
                self.iterable_dataset = iter(self.dataset)
                s = next(self.iterable_dataset)[self.tokens_column]
            tokens = torch.tensor(
                s,
                dtype=torch.long,
                device=device,
                requires_grad=False,
            )
            if (
                not self.prepend_bos
                and tokens[0] == self.model.tokenizer.bos_token_id  # type: ignore
            ):
                tokens = tokens[1:]
        self.n_dataset_processed += 1
        return tokens

## Residual stream

In [19]:
L1_coeff = 1e-1
training_tokens = 5_000_000
runner_cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distribution)
    model_name = "case3",
    model_class_name = "HookedTransformer",
    hook_name = "blocks.1.hook_resid_pre",
    hook_eval = "NOT_IN_USE",
    hook_layer = 1,
    hook_head_index = None,
    dataset_path = "",
    dataset_trust_remote_code = False,
    streaming = False,
    is_dataset_tokenized = True,
    context_size = 5,
    use_cached_activations = False,
    cached_activations_path = None,  # Defaults to "activations/{dataset}/{model}/{full_hook_name}_{hook_head_index}"

    # SAE Parameters
    d_in = model.cfg.d_model,
    d_sae = None,
    b_dec_init_method = "geometric_median",
    expansion_factor = 4,
    activation_fn = "relu",  # relu, tanh-relu
    normalize_sae_decoder = True,
    noise_scale = 0.0,
    from_pretrained_path = None,
    apply_b_dec_to_input = False,
    decoder_orthogonal_init = False,
    decoder_heuristic_init = False,
    init_encoder_as_decoder_transpose = False,

    # Activation Store Parameters
    n_batches_in_buffer = hf_dataset.shape[0],
    training_tokens = training_tokens,
    finetuning_tokens = 0,
    store_batch_size_prompts = 5,
    normalize_activations = "none",  # none, expected_average_only_in (Anthropic April Update), constant_norm_rescale (Anthropic Feb Update)

    # Misc
    device = device,
    act_store_device = "with_model",  # will be set by post init if with_model
    seed = 42,
    dtype = "float32",  # type: ignore #
    prepend_bos = False,

    # Performance - see compilation section of lm_runner.py for info
    autocast = False,  # autocast to autocast_dtype during training
    autocast_lm = False,  # autocast lm during activation fetching
    compile_llm = False,  # use torch.compile on the LLM
    llm_compilation_mode = None,  # which torch.compile mode to use
    compile_sae = False,  # use torch.compile on the SAE
    sae_compilation_mode = None,

    # Training Parameters

    ## Batch size
    train_batch_size_tokens = 5*100,

    ## Adam
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,

    ## Loss Function
    mse_loss_normalization = None,
    l1_coefficient = L1_coeff,
    lp_norm = 1,
    scale_sparsity_penalty_by_decoder_norm = False,
    l1_warm_up_steps = 0,

    ## Learning Rate Schedule
    lr = 3e-4,
    lr_scheduler_name = "constant",  # constant, cosineannealing, cosineannealingwarmrestarts
    lr_warm_up_steps = 0,
    lr_end = None,  # only used for cosine annealing, default is lr / 10
    lr_decay_steps = 0,
    n_restart_cycles = 1,  # used only for cosineannealingwarmrestarts

    ## FineTuning
    finetuning_method = None,  # scale, decoder or unrotated_decoder

    # Resampling protocol args
    use_ghost_grads = True,  # want to change this to true on some timeline.
    feature_sampling_window = 2000,
    dead_feature_window = 1000,  # unless this window is larger feature sampling,
    dead_feature_threshold = 1e-8,

    # Evals
    n_eval_batches = 10,
    eval_batch_size_prompts = None,  # useful if evals cause OOM

    # WANDB
    log_to_wandb = True,
    log_activations_store_to_wandb = False,
    log_optimizer_state_to_wandb = False,
    wandb_project = "benchmark_saes",
    wandb_id = None,
    run_name = None,
    wandb_entity = None,
    wandb_log_frequency = 10,
    eval_every_n_wandb_logs = 100000000000, # Make this a really big number; currently fails because it tries to compute CE loss.
    # Misc
    resume = False,
    n_checkpoints = 5,
    checkpoint_path = "checkpoints",
    verbose = True,
    model_kwargs = dict(),
    model_from_pretrained_kwargs = dict(),
    sae_lens_version = str(sae_lens.__version__),
    sae_lens_training_version = str(sae_lens.__version__),
)


Run name: 48-L1-0.1-LR-0.0003-Tokens-5.000e+06
n_tokens_per_buffer (millions): 0.032
Lower bound: n_contexts_per_buffer (millions): 0.0064
Total training steps: 10000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 5.0
n_tokens_per_dead_feature_window (millions): 2.5
We will reset the sparsity calculation 5 times.
Number tokens in sparsity calculation window: 1.00e+06
Using Ghost Grads.


In [39]:
from typing import Any, cast

model.tokenizer = hf_tokenizer
store = RepeatActivationsStore.from_config(model, runner_cfg, dataset=hf_dataset)
sae = TrainingSAE(runner_cfg)
trainer = SAETrainer(model, sae, store, save_checkpoint, cfg = runner_cfg)

if runner_cfg.log_to_wandb:
    wandb.init(
        project=runner_cfg.wandb_project,
        config=cast(Any, runner_cfg),
        name=runner_cfg.run_name,
        id=runner_cfg.wandb_id,
    )
trainer.fit()

wandb.finish()

200| MSE Loss 0.078 | L1 0.394:   2%|▋                                   | 100000/5000000 [00:18<14:06, 5786.03it/s]

KeyboardInterrupt: 

## MLP

In [21]:
L1_coeff = 1e-1
training_tokens = 5_000_000
runner_cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distribution)
    model_name = "case3",
    model_class_name = "HookedTransformer",
    hook_name = "blocks.0.hook_mlp_out",
    hook_eval = "NOT_IN_USE",
    hook_layer = 0,
    hook_head_index = None,
    dataset_path = "",
    dataset_trust_remote_code = False,
    streaming = False,
    is_dataset_tokenized = True,
    context_size = 5,
    use_cached_activations = False,
    cached_activations_path = None,  # Defaults to "activations/{dataset}/{model}/{full_hook_name}_{hook_head_index}"

    # SAE Parameters
    d_in = model.cfg.d_model,
    d_sae = None,
    b_dec_init_method = "geometric_median",
    expansion_factor = 4,
    activation_fn = "relu",  # relu, tanh-relu
    normalize_sae_decoder = True,
    noise_scale = 0.0,
    from_pretrained_path = None,
    apply_b_dec_to_input = False,
    decoder_orthogonal_init = False,
    decoder_heuristic_init = False,
    init_encoder_as_decoder_transpose = False,

    # Activation Store Parameters
    n_batches_in_buffer = hf_dataset.shape[0],
    training_tokens = training_tokens,
    finetuning_tokens = 0,
    store_batch_size_prompts = 5,
    normalize_activations = "none",  # none, expected_average_only_in (Anthropic April Update), constant_norm_rescale (Anthropic Feb Update)

    # Misc
    device = device,
    act_store_device = "with_model",  # will be set by post init if with_model
    seed = 42,
    dtype = "float32",  # type: ignore #
    prepend_bos = False,

    # Performance - see compilation section of lm_runner.py for info
    autocast = False,  # autocast to autocast_dtype during training
    autocast_lm = False,  # autocast lm during activation fetching
    compile_llm = False,  # use torch.compile on the LLM
    llm_compilation_mode = None,  # which torch.compile mode to use
    compile_sae = False,  # use torch.compile on the SAE
    sae_compilation_mode = None,

    # Training Parameters

    ## Batch size
    train_batch_size_tokens = 5*100,

    ## Adam
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,

    ## Loss Function
    mse_loss_normalization = None,
    l1_coefficient = L1_coeff,
    lp_norm = 1,
    scale_sparsity_penalty_by_decoder_norm = False,
    l1_warm_up_steps = 0,

    ## Learning Rate Schedule
    lr = 3e-4,
    lr_scheduler_name = "constant",  # constant, cosineannealing, cosineannealingwarmrestarts
    lr_warm_up_steps = 0,
    lr_end = None,  # only used for cosine annealing, default is lr / 10
    lr_decay_steps = 0,
    n_restart_cycles = 1,  # used only for cosineannealingwarmrestarts

    ## FineTuning
    finetuning_method = None,  # scale, decoder or unrotated_decoder

    # Resampling protocol args
    use_ghost_grads = True,  # want to change this to true on some timeline.
    feature_sampling_window = 2000,
    dead_feature_window = 1000,  # unless this window is larger feature sampling,
    dead_feature_threshold = 1e-8,

    # Evals
    n_eval_batches = 10,
    eval_batch_size_prompts = None,  # useful if evals cause OOM

    # WANDB
    log_to_wandb = True,
    log_activations_store_to_wandb = False,
    log_optimizer_state_to_wandb = False,
    wandb_project = "benchmark_saes",
    wandb_id = None,
    run_name = None,
    wandb_entity = None,
    wandb_log_frequency = 10,
    eval_every_n_wandb_logs = 100000000000, # Make this a really big number; currently fails because it tries to compute CE loss.
    # Misc
    resume = False,
    n_checkpoints = 5,
    checkpoint_path = "checkpoints",
    verbose = True,
    model_kwargs = dict(),
    model_from_pretrained_kwargs = dict(),
    sae_lens_version = str(sae_lens.__version__),
    sae_lens_training_version = str(sae_lens.__version__),
)


Run name: 48-L1-0.1-LR-0.0003-Tokens-5.000e+06
n_tokens_per_buffer (millions): 0.032
Lower bound: n_contexts_per_buffer (millions): 0.0064
Total training steps: 10000
Total wandb updates: 1000
n_tokens_per_feature_sampling_window (millions): 5.0
n_tokens_per_dead_feature_window (millions): 2.5
We will reset the sparsity calculation 5 times.
Number tokens in sparsity calculation window: 1.00e+06
Using Ghost Grads.


In [22]:
import shutil

In [23]:
from typing import Any, cast

model.tokenizer = hf_tokenizer
store = RepeatActivationsStore.from_config(model, runner_cfg, dataset=hf_dataset)
sae = TrainingSAE(runner_cfg)
trainer = SAETrainer(model, sae, store, save_checkpoint, cfg = runner_cfg)

if runner_cfg.log_to_wandb:
    wandb.init(
        project=runner_cfg.wandb_project,
        config=cast(Any, runner_cfg),
        name=runner_cfg.run_name,
        id=runner_cfg.wandb_id,
    )
trainer.fit()

wandb.finish()

wandb: Currently logged in as: evanhanders. Use `wandb login --relogin` to force relogin


2000| MSE Loss 0.014 | L1 0.095:  20%|██████▊                           | 1000000/5000000 [02:24<09:25, 7073.50it/s]

saving checkpoints/3zuzrmva/1000500


4000| MSE Loss 0.008 | L1 0.080:  40%|█████████████▌                    | 2000000/5000000 [04:45<06:50, 7304.02it/s]

saving checkpoints/3zuzrmva/2000500


...

saving checkpoints/3zuzrmva/4000500


10000| MSE Loss 0.005 | L1 0.079: 100%|█████████████████████████████████| 5000000/5000000 [11:55<00:00, 7107.40it/s]

saving checkpoints/3zuzrmva/final_5000000


10000| MSE Loss 0.005 | L1 0.079: 100%|█████████████████████████████████| 5000000/5000000 [11:55<00:00, 6985.69it/s]


details/current_l1_coefficient,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/current_learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
details/n_training_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
losses/auxiliary_reconstruction_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/ghost_grad_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/l1_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/mse_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/overall_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/explained_variance,▁███████████████████████████████████████
metrics/explained_variance_std,█▂▁▁▁▂▂▁▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/l0,█▆▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


# Inference

In [25]:
import wandb

# Initialize the wandb API
api = wandb.Api()



# Get the artifact from the old run
artifact = api.artifact('evanhanders/benchmark_saes/sae_case3_blocks.0.hook_mlp_out_48:latest')

# Download the artifact to a specified directory
artifact_mlp_out = artifact.download("./mlp_out_0")


artifact = api.artifact('evanhanders/benchmark_saes/sae_case3_blocks.1.hook_resid_pre_48:latest')

# Download the artifact to a specified directory
artifact_residual_1 = artifact.download("./hook_resid_pre_1")

wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  


In [29]:
from sae_lens import SAE

sae_mlp = SAE.load_from_pretrained(artifact_mlp_out, device=device)
sae_residual = SAE.load_from_pretrained(artifact_residual_1, device=device)

In [43]:
model.cfg.tokenizer_prepends_bos=False
model.cfg.default_prepend_bos=False

In [49]:
tokens = torch.Tensor(hf_dataset['tokens']).to(int)
logits, cache = model.run_with_cache(tokens, prepend_bos=False)

In [63]:
activations_mlp = sae_mlp.encode(cache[sae_mlp.cfg.hook_name])
activations_resid_1 = sae_residual.encode(cache[sae_residual.cfg.hook_name])

In [73]:
def get_top_k(activations, k=20):
    # Reshape
    batch, ctx, feat = activations.shape
    reshaped = activations.view(batch * ctx, feat)
    
    # Get the top-k samples and their indices
    top_samples = torch.topk(reshaped, dim=0, k=k)
    top_values = top_samples.values
    top_indices = top_samples.indices
    
    # print(top_indices)  # Print the shape of the top-k values
    
    # Compute the original batch and ctx positions
    original_batch_indices = top_indices // ctx
    original_ctx_indices = top_indices % ctx
    
    return top_values.cpu(), original_batch_indices.cpu(), original_ctx_indices.cpu()

# Example usage
top_values, batch_indices, ctx_indices = get_top_k(activations_mlp)
print("Top values shape:", top_values.shape)
print("Batch indices shape:", batch_indices.shape)
print("Context indices shape:", ctx_indices.shape)

print(activations_mlp.shape)
# print(batch_indices)

Top values shape: torch.Size([20, 48])
Batch indices shape: torch.Size([20, 48])
Context indices shape: torch.Size([20, 48])
torch.Size([1280, 5, 48])


In [124]:
feat = 16
k = 20

#feat 1 fires on a sequence where the first 3 are x.
#feat 2 fires on a b at position 1.
#feat 6 fires on an x at position 4.
#feat 7 fires weakly
#feat 12 fires on x at position 1.
#feat 16 fires on b at position 1.
print(f'feature {feat}')

activations = top_values[:,feat]
best_tokens = tokens[batch_indices[:,feat]]
print(ctx_indices[:,0].shape, best_tokens.shape)
print(active_token.shape)

active_idx = ctx_indices[:,feat]
active_token = best_tokens[range(k), active_idx]
active_tokens = hf_tokenizer.decode(active_token)
full_strs = [hf_tokenizer.decode(t) for t in best_tokens]

info = [
    {
        'tokens' : toks,
        'activation' : act.item(),
        'tok_idx' : idx.item(),
    }
    for act, idx, toks in zip(activations, active_idx, full_strs)
]

import pandas as pd
from IPython.display import display, HTML

# Create DataFrame
df = pd.DataFrame(info)

df.head(10)

feature 16
torch.Size([20]) torch.Size([20, 5])
torch.Size([20])


,tokens,activation,tok_idx
0,BOS b c b b,0.09,1
1,BOS b c a a,0.09,1
2,BOS b c x c,0.09,1
3,BOS b a c x,0.09,1
4,BOS b a a x,0.09,1
5,BOS b c b a,0.09,1
6,BOS b c x c,0.09,1
7,BOS b b b a,0.09,1
8,BOS b c x c,0.09,1
9,BOS b c c a,0.09,1


In [128]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Define the dropdown menu for 'feat'
feat_dropdown = widgets.Dropdown(
    options=range(sae_mlp.cfg.d_sae),
    value=16,
    description='Feature:',
)

# Function to update the DataFrame based on the selected feature
def update_dataframe(feat):
    k = 20
    
    print(f'feature {feat}')
    
    activations = top_values[:, feat]
    best_tokens = tokens[batch_indices[:, feat]]
    active_idx = ctx_indices[:, feat]
    active_token = best_tokens[range(k), active_idx]
    active_tokens = hf_tokenizer.decode(active_token)
    full_strs = [hf_tokenizer.decode(t) for t in best_tokens]
    
    info = [
        {
            'tokens': toks,
            'activation': act.item(),
            'tok_idx': idx.item(),
        }
        for act, idx, toks in zip(activations, active_idx, full_strs)
    ]
    
    # Create DataFrame
    df = pd.DataFrame(info)
    display(df.head(10))

# Create an interactive output widget
output = widgets.interactive_output(update_dataframe, {'feat': feat_dropdown})

# Display the dropdown menu and output
display(feat_dropdown, output)

Dropdown(description='Feature:', index=16, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …

Output()

MLP-0 features:
1 - on tok 4 when 1-3 are all x
2 - weak 'b' on tok 1
6 - 'x' on pos 2
7 - very weak, pos 4, when there are no x's?
12 - 'x' on pos 1
16 - weak 'b' on pos 1.
18 - BOS
19 - Strong fire on x on tok 4
20 - weak c on tok 3
21 - BOS
22 - weak c on tok 2
23 - weak 'x' on pos 4 or 2.
26 - BOS
28 - BOS
30 - weak 'c' on tok 3
33 - 'x' on tok 3.
34 - 'x' on tok 1
39 - 'b' on tok 3
40 - 'x' on tok 2
42 - 'x' on tok 3
43 - BOS
46 - 'c' on tok 1
